# Decision Trees INCLASS using Scikitlearn
Easy setup,works with categories as well as numbers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Set max rows and columns displayed in jupyter
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

#the following gives access to utils folder
#where utils package stores shared code
import os
import sys
PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.getcwd(),
                  os.pardir)
)

#only add it once
if (PROJECT_ROOT not in sys.path):
    sys.path.append(PROJECT_ROOT)

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

## Get the iris dataset, then train a decision tree on it and then see how accurate it is

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
dfi = pd.DataFrame(iris.data, columns=iris.feature_names)

mapper={v:i for v,i in enumerate(iris.target_names)}
def fun(x):
    return mapper[x]
vfunc=np.vectorize(fun)
dfi['target']=vfunc(iris.target)
dfi.head()
